In [ ]:
#| default_exp utils_polars_mapper

# 🐻‍❄️ Data Transforms

> High-performance JSON-to-database pipeline using Polars vectorized transformations.

In [ ]:
#| export

from __future__ import annotations
import polars as pl
from sqlalchemy import create_engine, text
from typing import Dict, List, Optional
import uuid
import logging
from nbdev.showdoc import show_doc


logger = logging.getLogger(__name__)

## 🎯 Overview

| Function | Purpose |
|----------|---------|
| `map_and_upsert` | Bulk JSON→DB upsert via staging table |
| `apply_schema` | Type conversions (dates, booleans, numbers) |

---

## 🏗️ Architecture

```
┌─────────────────────────────────────────────────────────────────┐
│                    Staging Table Pattern                        │
├─────────────────────────────────────────────────────────────────┤
│  1. df.write_database(staging_table) → fast bulk INSERT        │
│  2. INSERT ... ON CONFLICT SELECT FROM staging → vectorized    │
│  3. DROP TABLE staging → cleanup                                │
│                                                                 │
│  ⚡ 10-100x faster than row-by-row upserts                     │
└─────────────────────────────────────────────────────────────────┘
```

## 📥 Map & Upsert

| Parameter | Description |
|-----------|-------------|
| `df` | Polars DataFrame from JSON |
| `table_name` | Target database table (must exist) |
| `key_col` | Primary key for ON CONFLICT resolution |
| `db_uri` | Database connection string |
| `column_map` | Optional rename map `{json_col: db_col}` |
| `unnest_cols` | Optional list of nested columns to flatten |
| `type_map` | Optional type casting map `{col: pl.DataType}` |

**Returns:** `int` - Number of rows affected by the upsert operation

**Staging Table Pattern (10-100x faster than row-by-row):**
1. Write to temporary staging table (fast bulk insert)
2. Execute `INSERT ... ON CONFLICT` (database-native, vectorized)
3. Drop staging table (cleanup)

**Type Casting with `type_map`:**

When API data contains `None` values, Polars may infer incorrect types (e.g., `String` 
instead of `Int64`). This causes PostgreSQL type mismatch errors like:

```
column "balance" is of type integer but expression is of type text
```

Use `type_map` to explicitly cast columns before writing:

```python
rows = map_and_upsert(
    df=df,
    table_name='accounts',
    key_col='id',
    db_uri=db_uri,
    type_map={
        'balance_current': pl.Int64,
        'balance_available': pl.Int64,
        'balance_limit': pl.Int64
    }
)
print(f"Upserted {rows} rows")
```

**Basic Example:**

```python
import polars as pl

# JSON from API
json_data = [
    {'user_id_val': 1, 'ABC_1': 'Alice', 'extra': 'ignore'},
    {'user_id_val': 2, 'ABC_1': 'Bob', 'extra': 'ignore'}
]

df = pl.DataFrame(json_data)

rows_affected = map_and_upsert(
    df=df,
    table_name='users',
    key_col='user_id',
    db_uri='sqlite:///app.db',
    column_map={'user_id_val': 'user_id', 'ABC_1': 'name'}
)
print(f"Upserted {rows_affected} users")
```

In [ ]:
#| export

def map_and_upsert(
    df: pl.DataFrame, # The raw Polars DataFrame from JSON
    table_name: str, # Target database table name
    key_col: str, # Primary key column for conflict resolution
    db_uri: str, # SQLAlchemy connection string (e.g., 'sqlite:///db.db' or 'postgresql://...')
    column_map: dict = None, # Optional rename map {json_key: db_col}
    unnest_cols: list[str] = None, # List of Struct columns to flatten
    type_map: dict = None # Optional type casting map {col_name: pl.DataType}
) -> int:
    """
    Map JSON data to database columns and upsert using staging table pattern.
    
    **Returns:** Number of rows affected by the upsert operation
    
    **Type Casting:**
    When columns have `None` values, Polars may infer incorrect types (e.g., String
    instead of Int64). This causes PostgreSQL type mismatch errors. Use `type_map`
    to explicitly cast columns before writing to the database.
    """
    # Step 1: Rename columns if mapping provided
    if column_map:
        df = df.rename(column_map)
        logger.info(f"Renamed columns: {column_map}")
    
    # Step 2: Flatten nested columns if specified
    if unnest_cols:
        for col in unnest_cols:
            if col in df.columns:
                df = df.unnest(col)
                logger.info(f"Unnested column: {col}")
    
    # Step 3: Apply type casting if specified
    if type_map:
        cast_count = 0
        for col_name, dtype in type_map.items():
            if col_name not in df.columns:
                logger.warning(f"Column '{col_name}' in type_map not found in DataFrame, skipping")
                continue
            try:
                df = df.with_columns(pl.col(col_name).cast(dtype, strict=False))
                cast_count += 1
                logger.debug(f"Cast column '{col_name}' to {dtype}")
            except Exception as e:
                logger.warning(f"Failed to cast column '{col_name}' to {dtype}: {e}")
        logger.info(f"Applied type casting to {cast_count}/{len(type_map)} columns")
    
    # Step 4: Select only columns that exist in target table (drop extras)
    # This prevents errors from extra JSON fields
    engine = create_engine(db_uri)
    
    # Get target table columns
    with engine.connect() as conn:
        result = conn.execute(text(f"SELECT * FROM {table_name} LIMIT 0"))
        target_columns = list(result.keys())
    
    # Filter DataFrame to only target columns
    available_cols = [col for col in target_columns if col in df.columns]
    df = df.select(available_cols)
    logger.info(f"Selected columns for {table_name}: {available_cols}")
    
    # Step 5: Generate unique staging table name
    staging_table = f"staging_{uuid.uuid4().hex[:8]}"
    rows_affected = 0
    
    try:
        # Step 6: Write to staging table (fast bulk insert)
        df.write_database(
            table_name=staging_table,
            connection=db_uri,
            if_table_exists='replace'
        )
        logger.info(f"Wrote {len(df)} rows to staging table {staging_table}")
        
        # Step 7: Determine database type for dialect-specific SQL
        is_sqlite = 'sqlite' in db_uri.lower()
        
        # Step 8: Execute upsert from staging to target
        with engine.connect() as conn:
            if is_sqlite:
                # SQLite: INSERT OR REPLACE
                cols_str = ', '.join(available_cols)
                upsert_sql = f"""
                    INSERT OR REPLACE INTO {table_name} ({cols_str})
                    SELECT {cols_str} FROM {staging_table}
                """
            else:
                # PostgreSQL: INSERT ... ON CONFLICT DO UPDATE
                cols_str = ', '.join(available_cols)
                update_cols = [col for col in available_cols if col != key_col]
                update_set = ', '.join([f"{col} = EXCLUDED.{col}" for col in update_cols])
                
                upsert_sql = f"""
                    INSERT INTO {table_name} ({cols_str})
                    SELECT {cols_str} FROM {staging_table}
                    ON CONFLICT ({key_col}) DO UPDATE SET {update_set}
                """
            
            result = conn.execute(text(upsert_sql))
            rows_affected = result.rowcount if result.rowcount >= 0 else len(df)
            conn.commit()
            logger.info(f"Upserted {rows_affected} rows into {table_name}")
    
    finally:
        # Step 9: Cleanup - drop staging table
        with engine.connect() as conn:
            conn.execute(text(f"DROP TABLE IF EXISTS {staging_table}"))
            conn.commit()
            logger.debug(f"Dropped staging table {staging_table}")
    
    return rows_affected

In [ ]:
show_doc(map_and_upsert)

---

[source](https://github.com/abhisheksreesaila/fh-saas/blob/main/fh_saas/utils_polars_mapper.py#L21){target="_blank" style="float:right; font-size:smaller"}

### map_and_upsert

>      map_and_upsert (df:polars.dataframe.frame.DataFrame, table_name:str,
>                      key_col:str, db_uri:str, column_map:dict=None,
>                      unnest_cols:list[str]=None, type_map:dict=None)

*Map JSON data to database columns and upsert using staging table pattern.*

**Returns:** Number of rows affected by the upsert operation

**Type Casting:**
When columns have `None` values, Polars may infer incorrect types (e.g., String
instead of Int64). This causes PostgreSQL type mismatch errors. Use `type_map`
to explicitly cast columns before writing to the database.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | DataFrame |  | The raw Polars DataFrame from JSON |
| table_name | str |  | Target database table name |
| key_col | str |  | Primary key column for conflict resolution |
| db_uri | str |  | SQLAlchemy connection string (e.g., 'sqlite:///db.db' or 'postgresql://...') |
| column_map | dict | None | Optional rename map {json_key: db_col} |
| unnest_cols | list | None | List of Struct columns to flatten |
| type_map | dict | None | Optional type casting map {col_name: pl.DataType} |
| **Returns** | **int** |  |  |

## 🔧 Schema Transformations

| Parameter | Description |
|-----------|-------------|
| `df` | Polars DataFrame |
| `type_map` | Dict mapping column names to Polars dtypes |

**Supported conversions:**

| Type | Handling |
|------|----------|
| `pl.Date` | Parses `YYYY-MM-DD` strings |
| `pl.Datetime` | Parses datetime strings |
| `pl.Boolean` | Converts `"true"`/`"false"` strings |
| Other | Uses `cast()` (works for numeric types) |

**Example:**

```python
df = pl.DataFrame({
    'created_at': ['2024-01-15', '2024-01-16'],
    'is_active': ['true', 'false'],
    'amount': ['123.45', '678.90']
})

df = apply_schema(df, {
    'created_at': pl.Date,
    'is_active': pl.Boolean,
    'amount': pl.Float64
})
```

In [ ]:
#| export

def apply_schema(
    df: pl.DataFrame, # Input DataFrame
    type_map: dict # Column name -> Polars dtype (e.g., {'created_at': pl.Date, 'is_active': pl.Boolean})
) -> pl.DataFrame:
    """Apply explicit type conversions to DataFrame columns."""
    conversions = []
    
    for col_name, dtype in type_map.items():
        if col_name not in df.columns:
            logger.warning(f"Column {col_name} not found in DataFrame, skipping")
            continue
        
        # Handle different type conversions
        if dtype == pl.Date:
            conversions.append(pl.col(col_name).str.strptime(pl.Date, "%Y-%m-%d").alias(col_name))
        elif dtype == pl.Datetime:
            conversions.append(pl.col(col_name).str.strptime(pl.Datetime).alias(col_name))
        elif dtype == pl.Boolean:
            # Handle "true"/"false" strings
            conversions.append(
                pl.col(col_name).str.to_lowercase().eq("true").alias(col_name)
            )
        else:
            # Cast to specified type (works for numeric types)
            conversions.append(pl.col(col_name).cast(dtype).alias(col_name))
    
    if conversions:
        df = df.with_columns(conversions)
        logger.info(f"Applied schema conversions to {len(type_map)} columns")
    
    return df

In [ ]:
show_doc(apply_schema)

---

[source](https://github.com/abhisheksreesaila/fh-saas/blob/main/fh_saas/utils_polars_mapper.py#L133){target="_blank" style="float:right; font-size:smaller"}

### apply_schema

>      apply_schema (df:polars.dataframe.frame.DataFrame, type_map:dict)

*Apply explicit type conversions to DataFrame columns.*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| df | DataFrame | Input DataFrame |
| type_map | dict | Column name -> Polars dtype (e.g., {'created_at': pl.Date, 'is_active': pl.Boolean}) |
| **Returns** | **DataFrame** |  |

In [ ]:
#| hide

import nbdev as nb
nb.nbdev_export()